In [40]:
import pandas as pd
import numpy as np
import datetime as dt

In [43]:
# read in csv file with glassdoor data collected from webscraper
df = pd.read_csv("glassdoor_jobs.csv")

In [44]:
# split company_name into company and rating a columns
df[["company", "rating"]] = df["company_name"].str.split("\n", expand=True)
df.drop(columns="company_name", inplace=True)
df = df.iloc[:, [11, 12, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]
df

,company,rating,location,job_title,salary,salary_range,job_description,size,founded,type_of_ownership,industry,sector,revenue
0,Rijksinstituut voor Volksgezondheid en Milieu ...,3.5,Bilthoven,Stagiair data science COVID-19 Surveillance CA...,NaN,€ 3K - € 4K (Schatting werkgever),Achtergrond:\nHet centrum EPI creëert inzicht ...,Onbekend,Onbekend,Onbekend/niet van toepassing,NaN,NaN,NaN
1,EIFFEL,4.1,Utrecht,Data Scientist,€ 3.400 /mnd (schatting),Salaris opgegeven door werknemer:€ 3K - € 4K,Met jouw diepgaande statistische kennis maak j...,501 tot 1000 werknemers,1992,Privébedrijf,Business consulting,Management en consulting,25-100 miljoen USD
2,ICT Group,3.8,Groningen,Data Scientist,NaN,€ 3K - € 4K (Schatting werkgever),In het kort\nAls Data Scientist bij OrangeNXT ...,1001 tot 5000 werknemers,1978,Zelfstandige,IT-ondersteuning,IT,100-500 miljoen USD
3,Capgemini,4.1,Utrecht,Stage Data Science,NaN,€ 3K - € 4K (Schatting werkgever),Uren: Fulltime\nWerkplek: Utrecht\nErvaring: S...,5001 tot 10000 werknemers,1967,Beursgenoteerd bedrijf,Ondernemingssoftware en netwerkoplossingen,IT,Meer dan 10 miljard USD
4,Ordina,4.1,Nieuwegein,Data Scientist,NaN,€ 3K - € 4K (Schatting werkgever),Wat wij bieden\neen groep gepassioneerde vakge...,1001 tot 5000 werknemers,1973,Beursgenoteerd bedrijf,IT-ondersteuning,IT,100-500 miljoen USD
5,a.s.r.,4.2,Utrecht,Data Scientist,€ 4.400 /mnd (schatting),Salaris opgegeven door werknemer:€ 3K - € 6K,Wil jij van begin tot eind betrokken zijn bij ...,1001 tot 5000 werknemers,1720,Privébedrijf,Verzekeringsmaatschappijen,Verzekeringen,Minder dan 1 miljoen USD
6,a.s.r.,4.2,Utrecht,Datawarehouse Ontwikkelaar en Data Analist Ver...,€ 5.550 /mnd (schatting),Salaris opgegeven door werknemer:€ 5K - € 7K,Wil jij in complexe beleggingsdata duiken en l...,1001 tot 5000 werknemers,1720,Privébedrijf,Verzekeringsmaatschappijen,Verzekeringen,Minder dan 1 miljoen USD
7,ICT Group,3.8,Barendrecht,Senior Data Scientist Computer Vision,NaN,€ 3K - € 4K (Schatting werkgever),In het kort\nAls Data Scientist bij OrangeNXT ...,1001 tot 5000 werknemers,1978,Zelfstandige,IT-ondersteuning,IT,100-500 miljoen USD
8,Heineken Nederland,4.1,Bunnik,Stage - Data Science,NaN,€ 3K - € 4K (Schatting werkgever),Data Science stage Vrumona - September 2022\n\...,10000+ medewerkers,1864,Privébedrijf,Productie van voedingsmiddelen en dranken,Industrie,Meer dan 10 miljard USD
9,NaN,NaN,NaN,NaN,NaN,€ 3K - € 4K (Schatting werkgever),NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
# remove rows with empty values
df = df.dropna()
df

,company,rating,location,job_title,salary,salary_range,job_description,size,founded,type_of_ownership,industry,sector,revenue
1,EIFFEL,4.1,Utrecht,Data Scientist,€ 3.400 /mnd (schatting),Salaris opgegeven door werknemer:€ 3K - € 4K,Met jouw diepgaande statistische kennis maak j...,501 tot 1000 werknemers,1992,Privébedrijf,Business consulting,Management en consulting,25-100 miljoen USD
5,a.s.r.,4.2,Utrecht,Data Scientist,€ 4.400 /mnd (schatting),Salaris opgegeven door werknemer:€ 3K - € 6K,Wil jij van begin tot eind betrokken zijn bij ...,1001 tot 5000 werknemers,1720,Privébedrijf,Verzekeringsmaatschappijen,Verzekeringen,Minder dan 1 miljoen USD
6,a.s.r.,4.2,Utrecht,Datawarehouse Ontwikkelaar en Data Analist Ver...,€ 5.550 /mnd (schatting),Salaris opgegeven door werknemer:€ 5K - € 7K,Wil jij in complexe beleggingsdata duiken en l...,1001 tot 5000 werknemers,1720,Privébedrijf,Verzekeringsmaatschappijen,Verzekeringen,Minder dan 1 miljoen USD


In [46]:
# add indicator for salary provided hourly
df["hourly"] = df["salary_range"].map(lambda x: 1 if "per uur" in x.lower() else 0)

# clean up salary range
salary_by_employee = df["salary_range"].map(lambda x: x.split(":")[1])
salary_by_employee_blank = salary_by_employee.map(lambda x: x.replace("€", "").replace("K",""))
salary_by_employee_blank_hr = salary_by_employee_blank.map(lambda x: x.lower().replace("per uur", ""))

# # add min, max and avg salaries
df[["min_salary", "max_salary"]] = salary_by_employee_blank_hr.str.split(" - ", expand=True)
df.loc[df['max_salary'].isna(), "max_salary"] = df["min_salary"]
df["min_salary"] = df["min_salary"].map(lambda x: float(x.replace(",", ".")))
df["max_salary"] = df["max_salary"].map(lambda x: float(x.replace(",", ".")))
df["avg_salary"] = (df.min_salary + df.max_salary) / 2
df


/var/folders/jg/1ck3yxvn1sn72cwsg5c615740000gn/T/ipykernel_6481/815264180.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["hourly"] = df["salary_range"].map(lambda x: 1 if "per uur" in x.lower() else 0)
/var/folders/jg/1ck3yxvn1sn72cwsg5c615740000gn/T/ipykernel_6481/815264180.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[["min_salary", "max_salary"]] = salary_by_employee_blank_hr.str.split(" - ", expand=True)
/var/folders/jg/1ck3yxvn1sn72cwsg5c615740000gn/T/ipykernel_6481/815264180.py:10: S

,company,rating,location,job_title,salary,salary_range,job_description,size,founded,type_of_ownership,industry,sector,revenue,hourly,min_salary,max_salary,avg_salary
1,EIFFEL,4.1,Utrecht,Data Scientist,€ 3.400 /mnd (schatting),Salaris opgegeven door werknemer:€ 3K - € 4K,Met jouw diepgaande statistische kennis maak j...,501 tot 1000 werknemers,1992,Privébedrijf,Business consulting,Management en consulting,25-100 miljoen USD,0,3.0,4.0,3.5
5,a.s.r.,4.2,Utrecht,Data Scientist,€ 4.400 /mnd (schatting),Salaris opgegeven door werknemer:€ 3K - € 6K,Wil jij van begin tot eind betrokken zijn bij ...,1001 tot 5000 werknemers,1720,Privébedrijf,Verzekeringsmaatschappijen,Verzekeringen,Minder dan 1 miljoen USD,0,3.0,6.0,4.5
6,a.s.r.,4.2,Utrecht,Datawarehouse Ontwikkelaar en Data Analist Ver...,€ 5.550 /mnd (schatting),Salaris opgegeven door werknemer:€ 5K - € 7K,Wil jij in complexe beleggingsdata duiken en l...,1001 tot 5000 werknemers,1720,Privébedrijf,Verzekeringsmaatschappijen,Verzekeringen,Minder dan 1 miljoen USD,0,5.0,7.0,6.0


In [47]:
# check value counts
df.location.value_counts()

Utrecht    3
Name: location, dtype: int64

In [48]:
# age of company
df["age"] = df["founded"].map(lambda x: dt.date.today().year - int(x))
df

/var/folders/jg/1ck3yxvn1sn72cwsg5c615740000gn/T/ipykernel_6481/761360882.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["age"] = df["founded"].map(lambda x: dt.date.today().year - int(x))


,company,rating,location,job_title,salary,salary_range,job_description,size,founded,type_of_ownership,industry,sector,revenue,hourly,min_salary,max_salary,avg_salary,age
1,EIFFEL,4.1,Utrecht,Data Scientist,€ 3.400 /mnd (schatting),Salaris opgegeven door werknemer:€ 3K - € 4K,Met jouw diepgaande statistische kennis maak j...,501 tot 1000 werknemers,1992,Privébedrijf,Business consulting,Management en consulting,25-100 miljoen USD,0,3.0,4.0,3.5,30
5,a.s.r.,4.2,Utrecht,Data Scientist,€ 4.400 /mnd (schatting),Salaris opgegeven door werknemer:€ 3K - € 6K,Wil jij van begin tot eind betrokken zijn bij ...,1001 tot 5000 werknemers,1720,Privébedrijf,Verzekeringsmaatschappijen,Verzekeringen,Minder dan 1 miljoen USD,0,3.0,6.0,4.5,302
6,a.s.r.,4.2,Utrecht,Datawarehouse Ontwikkelaar en Data Analist Ver...,€ 5.550 /mnd (schatting),Salaris opgegeven door werknemer:€ 5K - € 7K,Wil jij in complexe beleggingsdata duiken en l...,1001 tot 5000 werknemers,1720,Privébedrijf,Verzekeringsmaatschappijen,Verzekeringen,Minder dan 1 miljoen USD,0,5.0,7.0,6.0,302


In [49]:
# parse job description
df["python_yn"] = df["job_description"].map(lambda x: 1 if 'python' in x.lower() else 0)
df["r_studio_yn"] = df["job_description"].map(lambda x: 1 if 'r studio' in x.lower() or 'r-studio' in x.lower() else 0)
df["spark_yn"] = df["job_description"].map(lambda x: 1 if 'spark' in x.lower() else 0)
df["aws_yn"] = df["job_description"].map(lambda x: 1 if 'aws' in x.lower() else 0)
df["excel_yn"] = df["job_description"].map(lambda x: 1 if 'excel' in x.lower() else 0)
df["azure_yn"] = df["job_description"].map(lambda x: 1 if 'azure' in x.lower() else 0)
df["sql_yn"] = df["job_description"].map(lambda x: 1 if 'sql' in x.lower() else 0)

df

/var/folders/jg/1ck3yxvn1sn72cwsg5c615740000gn/T/ipykernel_6481/541884246.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["python_yn"] = df["job_description"].map(lambda x: 1 if 'python' in x.lower() else 0)
/var/folders/jg/1ck3yxvn1sn72cwsg5c615740000gn/T/ipykernel_6481/541884246.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["r_studio_yn"] = df["job_description"].map(lambda x: 1 if 'r studio' in x.lower() or 'r-studio' in x.lower() else 0)
/var/folders/jg/1ck3yxvn1sn72cwsg5c615740000gn/T/ip

,company,rating,location,job_title,salary,salary_range,job_description,size,founded,type_of_ownership,...,max_salary,avg_salary,age,python_yn,r_studio_yn,spark_yn,aws_yn,excel_yn,azure_yn,sql_yn
1,EIFFEL,4.1,Utrecht,Data Scientist,€ 3.400 /mnd (schatting),Salaris opgegeven door werknemer:€ 3K - € 4K,Met jouw diepgaande statistische kennis maak j...,501 tot 1000 werknemers,1992,Privébedrijf,...,4.0,3.5,30,1,0,0,0,0,1,1
5,a.s.r.,4.2,Utrecht,Data Scientist,€ 4.400 /mnd (schatting),Salaris opgegeven door werknemer:€ 3K - € 6K,Wil jij van begin tot eind betrokken zijn bij ...,1001 tot 5000 werknemers,1720,Privébedrijf,...,6.0,4.5,302,1,0,0,0,0,0,1
6,a.s.r.,4.2,Utrecht,Datawarehouse Ontwikkelaar en Data Analist Ver...,€ 5.550 /mnd (schatting),Salaris opgegeven door werknemer:€ 5K - € 7K,Wil jij in complexe beleggingsdata duiken en l...,1001 tot 5000 werknemers,1720,Privébedrijf,...,7.0,6.0,302,0,0,0,0,0,1,0


In [37]:
# write result to csv
df.to_csv("glassdoor_jobs_cleaned.csv", index=False)